# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
!pip install gmaps
!jupyter nbextension list
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import gmaps.datasets
# Import API key
from api_keys import g_key

Known nbextensions:
  config dir: C:\Users\cfing\.jupyter\nbconfig
    notebook section
      jupyter-gmaps/extension enabled 
  config dir: C:\Users\cfing\Anaconda3\etc\jupyter\nbconfig
    notebook section
      jupyter-js-widgets/extension enabled 
      jupyter-gmaps/extension enabled 


      - Validating: ok
      - Validating: ok
      - Validating: ok


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cleaned= ("weather_for_import.csv")
df= pd.read_csv(cleaned)
df

,Unnamed: 0,City,Lat,Long,Max Temperature,Humidity,Cloudiness %,Wind Speed,Country,Date
0,1,albany,42.60,-73.97,82.00,37.0,6.0,2.48,US,1.590172e+09
1,2,safranbolu,41.25,32.69,41.31,85.0,11.0,2.62,TR,1.590173e+09
2,3,cherskiy,68.75,161.30,28.67,95.0,88.0,15.50,RU,1.590172e+09
3,4,ushuaia,-54.80,-68.30,53.60,37.0,75.0,13.87,AR,1.590172e+09
4,5,hobart,-42.88,147.33,39.99,93.0,0.0,8.05,AU,1.590172e+09
...,...,...,...,...,...,...,...,...,...,...
560,612,sinnar,13.57,33.57,94.14,34.0,67.0,14.58,SD,1.590173e+09
561,613,sikandra rao,27.70,78.40,95.25,12.0,3.0,6.38,IN,1.590173e+09
562,614,odweyne,9.41,45.06,79.97,52.0,34.0,13.96,SO,1.590173e+09
563,615,esna,25.29,32.55,104.00,21.0,0.0,16.96,EG,1.590173e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:

from ipywidgets.embed import embed_minimal_html
gmaps.configure(api_key= g_key)
locations = df[['Lat', 'Long']]
weights= df['Humidity']

fig = gmaps.figure(map_type='TERRAIN', center=(0,0), zoom_level =3) 
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights,max_intensity=0,point_radius = 30)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:

hotel_df = df.loc[(df["Max Temperature"] <= 80) & (df["Max Temperature"] >= 70) &(df["Cloudiness %"] == 0) &(df["Wind Speed"] < 10)]



### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df["Hotel Name"]= ''

C:\Users\cfing\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [8]:
params= {"radius" : 5000, "types": "lodging", "key": g_key}

for index, row in hotel_df.iterrows():
    city = row["City"]
    lat= row["Lat"]
    lng= row["Long"]
    
    params["location"]= f'{lat},{lng}'
    url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response= requests.get(url, params= params)
    response= response.json()
    try:
        print(f'Finding lodging in {city}')
        print('-------------------------')
        hotel_df.loc[index,"Hotel Name"] = response["results"][0]["name"]
    except (KeyError, IndexError):
        print(f' No lodging was returned...')
        hotel_df.loc[index,"Hotel Name"] = "No Hotel Found"
        

Finding lodging in romitan
-------------------------


C:\Users\cfing\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Finding lodging in caravelas
-------------------------
Finding lodging in kaka
-------------------------
 No lodging was returned...
Finding lodging in namibe
-------------------------
Finding lodging in tete
-------------------------
Finding lodging in benguela
-------------------------
Finding lodging in suicheng
-------------------------
Finding lodging in san lawrenz
-------------------------
Finding lodging in mushie
-------------------------
 No lodging was returned...


In [9]:
hotel_df

,Unnamed: 0,City,Lat,Long,Max Temperature,Humidity,Cloudiness %,Wind Speed,Country,Date,Hotel Name
27,30,romitan,39.93,64.38,75.20,50.0,0.0,6.93,UZ,1.590173e+09,Hotel Temur
53,66,caravelas,-17.71,-39.25,78.53,74.0,0.0,5.82,BR,1.590173e+09,Pousada dos Navegantes
168,191,kaka,37.35,59.61,78.53,19.0,0.0,7.31,TM,1.590173e+09,No Hotel Found
312,344,namibe,-15.20,12.15,76.59,63.0,0.0,9.42,AO,1.590172e+09,Chik-Chik Namibe
392,437,tete,-16.16,33.59,75.20,50.0,0.0,5.82,MZ,1.590173e+09,Zambeze Paraíso Misterioso
405,451,benguela,-12.58,13.41,75.83,81.0,0.0,2.68,AO,1.590173e+09,Hotel Praia Morena
500,550,suicheng,33.90,117.93,70.00,54.0,0.0,5.01,CN,1.590173e+09,Junyue Hotel
526,578,san lawrenz,36.06,14.20,71.60,64.0,0.0,4.70,MT,1.590173e+09,San Andrea Hotel
544,596,mushie,-3.02,16.90,77.58,78.0,0.0,2.53,CD,1.590173e+09,No Hotel Found


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row

# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]


In [11]:
# Add marker layer ontop of heat map
locations2= hotel_df[["Lat", "Long"]]


hotel_layer= gmaps.symbol_layer(locations2, fill_color='red', info_box_content = hotel_info)
fig.add_layer(hotel_layer)

fig




Figure(layout=FigureLayout(height='420px'))